In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1010861,2021-02-18,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1010862,2021-02-19,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1010863,2021-02-20,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1010864,2021-02-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
27471,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27473,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27475,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27477,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27479,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
626634,2021-02-18,Arkansas,Arkansas,5001,1939,38.00,5000,Arkansas,AR,Arkansas,State,3017804
626636,2021-02-19,Arkansas,Arkansas,5001,1939,38.00,5000,Arkansas,AR,Arkansas,State,3017804
626638,2021-02-20,Arkansas,Arkansas,5001,1941,38.00,5000,Arkansas,AR,Arkansas,State,3017804
626640,2021-02-21,Arkansas,Arkansas,5001,1942,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1010861,2021-02-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1010862,2021-02-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1010863,2021-02-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1010864,2021-02-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-22') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
398,2021-02-22,Snohomish,Washington,30474,530.00,53061,WA,County,822083,3706.92,64.47
794,2021-02-22,Cook,Illinois,470391,9833.00,17031,IL,County,5150233,9133.39,190.92
1189,2021-02-22,Orange,California,259581,3848.00,6059,CA,County,3175692,8174.00,121.17
1583,2021-02-22,Maricopa,Arizona,506046,8833.00,4013,AZ,County,4485414,11282.04,196.93
1977,2021-02-22,Los Angeles,California,1181994,19904.00,6037,CA,County,10039107,11773.90,198.26
...,...,...,...,...,...,...,...,...,...,...,...
1010457,2021-02-22,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1010590,2021-02-22,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
1010692,2021-02-22,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1010790,2021-02-22,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
398,2021-02-22,Snohomish,Washington,30474,530.00,53061,WA,County,822083,3706.92,64.47,64.47,3706.92
794,2021-02-22,Cook,Illinois,470391,9833.00,17031,IL,County,5150233,9133.39,190.92,190.92,9133.39
1189,2021-02-22,Orange,California,259581,3848.00,6059,CA,County,3175692,8174.00,121.17,121.17,8174.00
1583,2021-02-22,Maricopa,Arizona,506046,8833.00,4013,AZ,County,4485414,11282.04,196.93,196.93,11282.04
1977,2021-02-22,Los Angeles,California,1181994,19904.00,6037,CA,County,10039107,11773.90,198.26,198.26,11773.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010457,2021-02-22,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1010590,2021-02-22,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
1010692,2021-02-22,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1010790,2021-02-22,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
712775,2021-02-22,Clarke,Alabama,3421,48.00,1025,AL,County,23622,14482.26,203.20,203.20,14482.26,1
655105,2021-02-22,Hale,Alabama,2074,64.00,1065,AL,County,14651,14156.03,436.83,436.83,14156.03,2
513081,2021-02-22,Lowndes,Alabama,1300,46.00,1085,AL,County,9726,13366.23,472.96,472.96,13366.23,3
455132,2021-02-22,Etowah,Alabama,13078,307.00,1055,AL,County,102268,12787.97,300.19,300.19,12787.97,4
513415,2021-02-22,Marengo,Alabama,2385,54.00,1091,AL,County,18863,12643.80,286.27,286.27,12643.80,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961521,2021-02-22,Platte,Wyoming,583,11.00,56031,WY,County,8393,6946.26,131.06,131.06,6946.26,19
625649,2021-02-22,Sublette,Wyoming,671,7.00,56035,WY,County,9831,6825.35,71.20,71.20,6825.35,20
863568,2021-02-22,Niobrara,Wyoming,148,2.00,56027,WY,County,2356,6281.83,84.89,84.89,6281.83,21
776542,2021-02-22,Lincoln,Wyoming,1212,11.00,56023,WY,County,19830,6111.95,55.47,55.47,6111.95,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
513081,2021-02-22,Lowndes,Alabama,1300,46.00,1085,AL,County,9726,13366.23,472.96,472.96,13366.23,3,1
655105,2021-02-22,Hale,Alabama,2074,64.00,1065,AL,County,14651,14156.03,436.83,436.83,14156.03,2,2
453792,2021-02-22,Clay,Alabama,1422,54.00,1027,AL,County,13235,10744.24,408.01,408.01,10744.24,22,3
553158,2021-02-22,Greene,Alabama,867,32.00,1063,AL,County,8111,10689.19,394.53,394.53,10689.19,23,4
218807,2021-02-22,Walker,Alabama,6451,240.00,1127,AL,County,63521,10155.70,377.83,377.83,10155.70,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551825,2021-02-22,Hot Springs,Wyoming,360,3.00,56017,WY,County,4413,8157.72,67.98,67.98,8157.72,16,19
757308,2021-02-22,Uinta,Wyoming,2030,12.00,56041,WY,County,20226,10036.59,59.33,59.33,10036.59,4,20
776542,2021-02-22,Lincoln,Wyoming,1212,11.00,56023,WY,County,19830,6111.95,55.47,55.47,6111.95,22,21
217784,2021-02-22,Teton,Wyoming,3331,9.00,56039,WY,County,23464,14196.22,38.36,38.36,14196.22,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,64.47,3706.92,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,64.47,3706.92,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,64.47,3706.92,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,64.47,3706.92,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,64.47,3706.92,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008474,2021-02-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1008475,2021-02-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1008476,2021-02-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1008477,2021-02-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
710061,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,203.20,14482.26,45,1,2.00
710062,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,203.20,14482.26,45,1,0.00
710063,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,203.20,14482.26,45,1,1.00
710064,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,203.20,14482.26,45,1,4.00
710065,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,203.20,14482.26,45,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789426,2021-02-18,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
789427,2021-02-19,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
789428,2021-02-20,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
789429,2021-02-21,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
511025,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13366.23,1,3,1.00,0.00
511026,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13366.23,1,3,0.00,0.00
511027,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13366.23,1,3,0.00,0.00
511028,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13366.23,1,3,0.00,0.00
511029,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13366.23,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509350,2021-02-18,Albany,Wyoming,3871,11.00,56001,WY,County,38880,9956.28,28.29,28.29,10012.86,23,6,0.00,0.00
509351,2021-02-19,Albany,Wyoming,3880,11.00,56001,WY,County,38880,9979.42,28.29,28.29,10012.86,23,6,9.00,0.00
509352,2021-02-20,Albany,Wyoming,3880,11.00,56001,WY,County,38880,9979.42,28.29,28.29,10012.86,23,6,0.00,0.00
509353,2021-02-21,Albany,Wyoming,3887,11.00,56001,WY,County,38880,9997.43,28.29,28.29,10012.86,23,6,7.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-22') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
261059,2021-02-22,Imperial,California,26840,583.00,6025,CA,County,181215,14811.14,321.72,321.72,14811.14,1,2,0.00,0.00
1977,2021-02-22,Los Angeles,California,1181994,19904.00,6037,CA,County,10039107,11773.90,198.26,198.26,11773.90,2,5,934.00,19.00
456424,2021-02-22,Inyo,California,1232,34.00,6027,CA,County,18039,6829.65,188.48,188.48,6829.65,3,29,6.00,0.00
58373,2021-02-22,Stanislaus,California,55347,931.00,6099,CA,County,550660,10051.03,169.07,169.07,10051.03,4,10,156.00,5.00
72330,2021-02-22,Tulare,California,47562,722.00,6107,CA,County,466195,10202.17,154.87,154.87,10202.17,5,9,163.00,2.00
27117,2021-02-22,Riverside,California,287822,3664.00,6065,CA,County,2470546,11650.14,148.31,148.31,11650.14,6,6,288.00,31.00
553100,2021-02-22,Kings,California,21881,216.00,6031,CA,County,152940,14306.92,141.23,141.23,14306.92,7,3,24.00,1.00
413769,2021-02-22,Merced,California,28655,391.00,6047,CA,County,277680,10319.43,140.81,140.81,10319.43,8,8,166.00,3.00
26411,2021-02-22,Fresno,California,94374,1367.00,6019,CA,County,999101,9445.89,136.82,136.82,9445.89,9,14,140.00,0.00
46475,2021-02-22,San Joaquin,California,66046,1039.00,6077,CA,County,762148,8665.77,136.33,136.33,8665.77,10,18,155.00,13.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
961072,2021-02-22,Lassen,California,5591,19.00,6035,CA,County,30573,18287.38,62.15,62.15,18287.38,36,1,33.00,0.00
261059,2021-02-22,Imperial,California,26840,583.00,6025,CA,County,181215,14811.14,321.72,321.72,14811.14,1,2,0.00,0.00
553100,2021-02-22,Kings,California,21881,216.00,6031,CA,County,152940,14306.92,141.23,141.23,14306.92,7,3,24.00,1.00
130255,2021-02-22,San Bernardino,California,284866,2637.00,6071,CA,County,2180085,13066.74,120.96,120.96,13066.74,13,4,227.00,25.00
1977,2021-02-22,Los Angeles,California,1181994,19904.00,6037,CA,County,10039107,11773.90,198.26,198.26,11773.90,2,5,934.00,19.00
27117,2021-02-22,Riverside,California,287822,3664.00,6065,CA,County,2470546,11650.14,148.31,148.31,11650.14,6,6,288.00,31.00
163665,2021-02-22,Kern,California,101849,783.00,6029,CA,County,900202,11314.02,86.98,86.98,11314.02,24,7,111.00,10.00
413769,2021-02-22,Merced,California,28655,391.00,6047,CA,County,277680,10319.43,140.81,140.81,10319.43,8,8,166.00,3.00
72330,2021-02-22,Tulare,California,47562,722.00,6107,CA,County,466195,10202.17,154.87,154.87,10202.17,5,9,163.00,2.00
58373,2021-02-22,Stanislaus,California,55347,931.00,6099,CA,County,550660,10051.03,169.07,169.07,10051.03,4,10,156.00,5.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
261059,2021-02-22,Imperial,California,26840,583.00,6025,CA,County,181215,14811.14,321.72,321.72,14811.14,1,2,0.00,0.00
1977,2021-02-22,Los Angeles,California,1181994,19904.00,6037,CA,County,10039107,11773.90,198.26,198.26,11773.90,2,5,934.00,19.00
456424,2021-02-22,Inyo,California,1232,34.00,6027,CA,County,18039,6829.65,188.48,188.48,6829.65,3,29,6.00,0.00
58373,2021-02-22,Stanislaus,California,55347,931.00,6099,CA,County,550660,10051.03,169.07,169.07,10051.03,4,10,156.00,5.00
72330,2021-02-22,Tulare,California,47562,722.00,6107,CA,County,466195,10202.17,154.87,154.87,10202.17,5,9,163.00,2.00
27117,2021-02-22,Riverside,California,287822,3664.00,6065,CA,County,2470546,11650.14,148.31,148.31,11650.14,6,6,288.00,31.00
553100,2021-02-22,Kings,California,21881,216.00,6031,CA,County,152940,14306.92,141.23,141.23,14306.92,7,3,24.00,1.00
413769,2021-02-22,Merced,California,28655,391.00,6047,CA,County,277680,10319.43,140.81,140.81,10319.43,8,8,166.00,3.00
26411,2021-02-22,Fresno,California,94374,1367.00,6019,CA,County,999101,9445.89,136.82,136.82,9445.89,9,14,140.00,0.00
46475,2021-02-22,San Joaquin,California,66046,1039.00,6077,CA,County,762148,8665.77,136.33,136.33,8665.77,10,18,155.00,13.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14811.14,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3412,02/18/21,Lassen,5558,19.00,18179.44,62.15,62.15,18287.38,36,1,12.00,0.00
3413,02/19/21,Lassen,5558,19.00,18179.44,62.15,62.15,18287.38,36,1,0.00,0.00
3414,02/20/21,Lassen,5558,19.00,18179.44,62.15,62.15,18287.38,36,1,0.00,0.00
3415,02/21/21,Lassen,5558,19.00,18179.44,62.15,62.15,18287.38,36,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14811.14,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3486,02/18/21,San Joaquin,65705,1021.00,8621.03,133.96,136.33,8665.77,10,18,46.00,3.00
3487,02/19/21,San Joaquin,65891,1026.00,8645.43,134.62,136.33,8665.77,10,18,186.00,5.00
3488,02/20/21,San Joaquin,65891,1026.00,8645.43,134.62,136.33,8665.77,10,18,0.00,0.00
3489,02/21/21,San Joaquin,65891,1026.00,8645.43,134.62,136.33,8665.77,10,18,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)